In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import keras
import tensorflow as tf
from net.style_autoencoder import StyleTransfer
IMAGES_FOLDER = '/mnt/s/CV/StyleTransferData/'
CONTENT_FOLDER = os.path.join(IMAGES_FOLDER, 'test2015')
STYLE_FOLDER = os.path.join(IMAGES_FOLDER, 'wikiart_styles')
BATCH_SIZE = 8
EPOCHS = 15
IMG_SIZE = (256, 256)
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
def delete_corrupted_imgs(path):
    
    def remove(file):
        print('=='*10)
        os.remove(os.path.join(path, file))
        print(f'Deleted corrupted {file}')
        
    files = os.listdir(path)
    for file in files:
        try:
            img = tf.io.decode_jpeg(tf.io.read_file(os.path.join(path, file)))
            if img is None:
                remove(file)
        except Exception as e:
            remove(file)

# delete_corrupted_imgs(STYLE_FOLDER)
# delete_corrupted_imgs(CONTENT_FOLDER)

In [3]:
dataset_config = dict(
    label_mode=None, 
    labels=None,
    shuffle=True,
    image_size=IMG_SIZE,
    batch_size=None,
    crop_to_aspect_ratio=True
)

content_dataset = keras.utils.image_dataset_from_directory(
    CONTENT_FOLDER,
    **dataset_config
)
style_dataset = keras.utils.image_dataset_from_directory(
    STYLE_FOLDER,
    **dataset_config
)

dataset = tf.data.Dataset.zip((content_dataset, style_dataset)).batch(BATCH_SIZE, drop_remainder=True)

Found 81434 files.
Found 79412 files.


In [4]:
model = StyleTransfer()
optimizer = keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate=1e-4,
        decay_steps=10,
        decay_rate=5e-5
    )
)
loss_fn = keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss_fn=loss_fn)
model.summary()

Model: "style_transfer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ada_in (AdaIN)                  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Encoder)               │ ?                      │     3,505,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │     3,505,219 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,010,947 (26.74 MB)

 Trainable params: 3,505,219 (13.37 MB)

 Non-trainable params: 3,505,728 (13.37 MB)

In [5]:
history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[
        keras.callbacks.ModelCheckpoint(
            './checkpoint/model_{epoch}.keras',
            monitor='total_loss',
            save_best_only=False,
            save_freq='epoch'
        )
    ]
)

Epoch 1/15


I0000 00:00:1710118354.633926    2179 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 899/9926 ━━━━━━━━━━━━━━━━━━━━ 14:36 97ms/step - content_loss: 706071.1250 - learning_rate: 9.9887e-05 - style_loss: 2570429.7500 - total_loss: 3276501.0000

Corrupt JPEG data: premature end of data segment


2254/9926 ━━━━━━━━━━━━━━━━━━━━ 12:13 96ms/step - content_loss: 610571.8750 - learning_rate: 9.9719e-05 - style_loss: 1694251.5000 - total_loss: 2304824.5000

...

8979/9926 ━━━━━━━━━━━━━━━━━━━━ 1:28 93ms/step - content_loss: 186191.5000 - learning_rate: 5.8617e-05 - style_loss: 193166.6562 - total_loss: 379358.0625

Corrupt JPEG data: premature end of data segment


9721/9926 ━━━━━━━━━━━━━━━━━━━━ 19s 93ms/step - content_loss: 186096.5312 - learning_rate: 5.8585e-05 - style_loss: 193099.5000 - total_loss: 379195.9062

Corrupt JPEG data: 419 extraneous bytes before marker 0xd9


9926/9926 ━━━━━━━━━━━━━━━━━━━━ 930s 93ms/step - content_loss: 186074.3906 - learning_rate: 5.8577e-05 - style_loss: 193084.4688 - total_loss: 379158.7500
